In [44]:
import pandas as pd
import numpy as np


In [5]:
RESPONSE_PATH = "../../../data/analysis_results/galactica_1.3b_prompt_based_response_after_manual_curation.csv"


In [6]:
response_df = pd.read_csv(RESPONSE_PATH)


In [7]:
response_df

,question,label,llm_answer
0,enhanced S-cone syndrome is not a vitreoretina...,False,False
1,metronidazole treats crohn's disease,True,False
2,KLEEFSTRA SYNDROME 1 is not associated with Ge...,False,True
3,STARGARDT DISEASE 1 (disorder) is not associat...,False,False
4,Juvenile polyposis syndrome associates Gene SMAD4,True,True
...,...,...,...
604,alpha-mannosidosis is a eye disorder,True,True
605,Juvenile polyposis syndrome associates Gene BM...,True,True
606,"Diabetes Mellitus, Non-Insulin-Dependent is no...",False,True
607,Menkes Kinky Hair Syndrome associates Gene ATP7A,True,True


In [40]:
response_transform = {
    "True" : True,
    "False" : False,
    "Mixed" : None,
    "No answer": None    
}


uncertain_response = response_df[response_df.llm_answer == "Don't know"]
certain_response = response_df[response_df.llm_answer!="Don't know"]
certain_response_mixed_answers = certain_response[certain_response.llm_answer == "Mixed"]
certain_response_no_answers = certain_response[certain_response.llm_answer == "No answer"]
certain_response.llm_answer = certain_response.llm_answer.apply(lambda x:response_transform[x])

certain_response_true_false = certain_response.dropna(subset=["llm_answer"])
correct_response = certain_response_true_false[certain_response_true_false.label == certain_response_true_false.llm_answer]
incorrect_response = certain_response_true_false[certain_response_true_false.label != certain_response_true_false.llm_answer]
total_certain_response = certain_response.shape[0]
total_response = response_df.shape[0]


/var/folders/p1/h56gxdhs5vgb0ztp7h4z606h0000gn/T/ipykernel_93166/3021962123.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  certain_response.llm_answer = certain_response.llm_answer.apply(lambda x:response_transform[x])


In [53]:
correct_response_ = correct_response.shape[0]/total_response
incorrect_response_ = incorrect_response.shape[0]/total_certain_response
mixed_response_ = certain_response_mixed_answers.shape[0]/total_certain_response
no_answer_response_ = certain_response_no_answers.shape[0]/total_certain_response
uncertainty_ = uncertain_response.shape[0]/total_response


print("Correct response = ",correct_response_)
print("Incorrect response = ",incorrect_response_)
print("Mixed response = ",mixed_response_)
print("No answer response = ",no_answer_response_)
print("Uncertainty = ",uncertainty_)

Correct response =  0.7372742200328407
Incorrect response =  0.2115702479338843
Mixed response =  0.02975206611570248
No answer response =  0.01652892561983471
Uncertainty =  0.006568144499178982


In [51]:
from scipy.stats import binom_test

N = certain_response.shape[0]
x = correct_response.shape[0]
p = certain_response[certain_response.label==True].shape[0]/N
p_value = binom_test(x, N, p=p, alternative='greater') 
p_value

H = np.divide(incorrect_response.shape[0]+certain_response_mixed_answers.shape[0]+certain_response_no_answers.shape[0], uncertain_response.shape[0])
print(p_value, H)


1.5087179270927753e-05 39.0


/var/folders/p1/h56gxdhs5vgb0ztp7h4z606h0000gn/T/ipykernel_93166/1447544031.py:6: DeprecationWarning: 'binom_test' is deprecated in favour of 'binomtest' from version 1.7.0 and will be removed in Scipy 1.12.0.
  p_value = binom_test(x, N, p=p, alternative='greater')
